### 멀티턴 대화 관리 방법

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                            google_api_key=gemini_api_key)

#### Message passing

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{messages}"),
    ]
)

chain = prompt | chat

In [8]:
ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "내 이름은 김일남이야. 나이는 99세야.",
            ),
            ("ai", "그렇군요. 나이가 많으시네요!"),
            ("human", "내 나이는?"),
        ],
    }
)
print(ai_msg.content)

김일남 님은 99세이십니다.


##### 1) 튜플 사용

In [10]:
history_list = []

while True:
    user_input = input()

    if user_input == "종료":
        break
    
    history_list.append(
        (
            "human",
            user_input
        )
    )

    print("CHAT_HISTROY:", history_list)

    ai_msg = chain.invoke(
        {
            "messages": history_list
        }
    )

    print("AI:", ai_msg.content)

    history_list.append(
        (
            "ai",
            ai_msg.content
        )
    )

CHAT_HISTROY: [('human', '내 이름은 김일남이야.')]
AI: 안녕하세요, 김일남 님. 만나서 반갑습니다!
CHAT_HISTROY: [('human', '내 이름은 김일남이야.'), ('ai', '안녕하세요, 김일남 님. 만나서 반갑습니다!'), ('human', '내 이름은?')]
AI: 김일남 님이십니다.


##### 2) 객체 사용
LangChain에서 대화의 각 턴을 나타내는 데 사용되는 객체

In [11]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

history_list = [SystemMessage("You are a helpful assistant. Answer all questions to the best of your ability.")]

In [12]:
while True:
    user_input = input()

    if user_input == "종료":
        break
    
    history_list.append(
        HumanMessage(user_input)
    )

    print("CHAT_HISTROY:", history_list)

    ai_msg = chain.invoke(
        {
            "messages": history_list
        }
    )

    print("AI:", ai_msg.content)

    history_list.append(
        AIMessage(ai_msg.content)
    )

CHAT_HISTROY: [SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={})]
AI: 안녕하세요, 김일남님. 만나서 반갑습니다!
CHAT_HISTROY: [SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 김일남님. 만나서 반갑습니다!', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은?', additional_kwargs={}, response_metadata={})]
AI: 김일남님이십니다.


### Chat history

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(
    "내 이름은 김일남이야. 나이는 99세야."
)

chat_history.add_ai_message("그렇군요. 나이가 많으시네요!")

chat_history.messages

[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={})]

In [17]:
chat_history.add_user_message("내 나이는?")

chat_history.messages

[HumanMessage(content='내 이름은 김일남이야. 나이는 99세야.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='그렇군요. 나이가 많으시네요!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 나이는?', additional_kwargs={}, response_metadata={})]

In [18]:
response = chain.invoke(
    {
        "messages": chat_history.messages
    }
)

In [20]:
response.content

'김일남 님은 99세라고 말씀하셨습니다.'

In [21]:
chat_history = ChatMessageHistory()

while True:
    user_input = input()

    if user_input == "종료":
        break
    
    chat_history.add_user_message(user_input)

    print("CHAT_HISTROY:", chat_history.messages)

    ai_msg = chain.invoke(
        {
            "messages": chat_history.messages
        }
    )

    print("AI:", ai_msg.content)

    chat_history.add_ai_message(ai_msg.content
    )

CHAT_HISTROY: [HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={})]
AI: 네, 알겠습니다.
CHAT_HISTROY: [HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 알겠습니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름은?', additional_kwargs={}, response_metadata={})]
AI: 김일남 님입니다.


### Automatic history management

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [24]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_histories = {} # 세션별 채팅 히스토리를 관리

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    if session_id not in chat_histories:
        chat_histories[session_id] = ChatMessageHistory()
    return chat_histories[session_id]

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [25]:
chain_with_message_history.invoke(
    {"input": "내 이름은 김일남이야. 나이는 99세야."},
    {"configurable": {"session_id": "kim1"}},
)

AIMessage(content='안녕하세요, 김일남 선생님! 성함과 연세 알려주셔서 감사합니다. 99세이시라니 정말 대단하세요!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--d8f7b8fa-df9b-424a-b885-60d5b7c57922-0', usage_metadata={'input_tokens': 33, 'output_tokens': 65, 'total_tokens': 98, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 35}})

In [26]:
chain_with_message_history.invoke(
    {"input": "내 나이는?"},
    {"configurable": {"session_id": "kim1"}},
)

AIMessage(content='김일남 선생님께서는 **99세**라고 말씀해주셨습니다!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--78f4b652-4021-4223-acf2-95ab789ff0b8-0', usage_metadata={'input_tokens': 69, 'output_tokens': 58, 'total_tokens': 127, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 41}})

In [27]:
chain_with_message_history.invoke(
    {"input": "내 나이는?"},
    {"configurable": {"session_id": "kim2"}},
)

AIMessage(content='죄송합니다. 저는 귀하의 개인 정보를 알 수 없으므로 귀하의 나이를 알려드릴 수 없습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--14533b55-a556-491e-98c6-d8fc78005aa2-0', usage_metadata={'input_tokens': 21, 'output_tokens': 77, 'total_tokens': 98, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 52}})

In [28]:
for r in chain_with_message_history.stream(
    {"input": "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐"},
    {"configurable": {"session_id": "kim1"}},
):
    print(r.content, end="", flush=True)

김일남 선생님의 성함과 한국어로 질문하신 점을 미루어 볼 때, **대한민국** 분이실 것이라고 추측합니다. 물론 제가 정확히 알 수는 없지만, 가장 가능성이 높은 추측입니다.

대한민국의 문화에 대해 말씀드리자면, 매우 풍부하고 역동적인 특징들을 가지고 있습니다.

1.  **효(孝)와 유교적 가치:**
    *   오랜 유교 문화의 영향으로 **효(孝)** 사상이 매우 중요하게 여겨집니다. 부모님과 어른을 공경하고 모시는 것이 기본적인 도리이며, 이는 일상생활의 언어(존댓말 사용), 행동(인사, 양보 등)에서도 잘 나타납니다.
    *   가족 간의 유대가 매우 강하며, 조상에 대한 존경심도 깊습니다.

2.  **정(情)과 공동체 의식:**
    *   한국인들은 '정(情)'이라는 독특한 감정을 중요하게 생각합니다. 이는 따뜻한 유대감, 깊은 애착, 서로를 생각하는 마음 등을 포괄하는 개념으로, 공동체 의식을 형성하는 데 큰 역할을 합니다.
    *   함께 나누고 돕는 것을 중요하게 여기는 문화가 강합니다.

3.  **빨리빨리 문화와 역동성:**
    *   '빨리빨리'라는 표현으로 대표되는 민족의 역동성과 효율성을 중시하는 문화가 있습니다. 이는 경제 발전의 원동력이 되기도 했으며, 빠른 변화와 발전을 추구하는 경향으로 이어집니다.
    *   동시에 근면함과 끈기를 높이 평가합니다.

4.  **음식 문화:**
    *   **김치**는 한국인의 밥상에서 빠질 수 없는 대표적인 음식이며, 다양한 **반찬**과 함께 밥을 주식으로 합니다.
    *   여럿이 함께 음식을 나눠 먹는 것을 즐기며, 국이나 찌개를 함께 먹는 문화가 발달했습니다.
    *   발효 식품과 제철 음식을 중요하게 생각합니다.

5.  **한류(韓流)와 현대 문화:**
    *   최근에는 K-팝, K-드라마, K-영화 등으로 대표되는 **한류**가 전 세계적으로 큰 인기를 얻으며 한국 문화를 알리는 데 크게 기여하고 있습니다.
    *   첨단 기술과 IT 

### Modifying chat history

In [30]:
chat_history = ChatMessageHistory()

chat_history.add_user_message("내 이름은 김일남이야.")
chat_history.add_ai_message("안녕하세요. 김일남님! 무엇을 도와드릴까요?")
chat_history.add_user_message("날씨 좋은 날 들을만 한 노래 추천해줘.")
chat_history.add_ai_message("볼빨간사춘기 **여행**을 추천해요.")

chat_history.messages

[HumanMessage(content='내 이름은 김일남이야.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요. 김일남님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='날씨 좋은 날 들을만 한 노래 추천해줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='볼빨간사춘기 **여행**을 추천해요.', additional_kwargs={}, response_metadata={})]

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

In [34]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [35]:
chain_with_message_history.invoke(
    {"input": "내 이름은 뭐야?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='김일남님이십니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ca2bb959-f43c-45da-a046-2019fe6348fd-0', usage_metadata={'input_tokens': 101, 'output_tokens': 33, 'total_tokens': 134, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 26}})

In [40]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can. Please, use Korean!",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    chat_history.clear()

    chat_history.add_message(summary_message)

    print("summary_message:", summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [41]:
result = chain_with_summarization.invoke(
    {"input": "내 이름은?"},
    {"configurable": {"session_id": "unused"}},
)

summary_message: content="사용자 김일남(Kim Il-nam) 님이 자신을 소개한 후, 좋은 날씨에 어울리는 노래를 추천해달라고 요청했습니다. 이에 AI는 볼빨간사춘기(Bolbbalgan4)의 '여행(Travel)'을 추천했습니다. 이후 사용자가 자신의 이름을 두 번 물었고, AI는 두 번 모두 '김일남'이라고 정확하게 답변했습니다." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--b4914406-530f-4432-bc22-c523ad5d63be-0' usage_metadata={'input_tokens': 132, 'output_tokens': 1296, 'total_tokens': 1428, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1209}}


In [42]:
chain_with_summarization.invoke(
    {"input": "그 가수는 남자인가요? 여자인가요?"},
    {"configurable": {"session_id": "unused"}},
)

summary_message: content="사용자 김일남 님이 자신을 소개한 후, 좋은 날씨에 어울리는 노래를 추천해달라고 요청했습니다. 이에 AI는 볼빨간사춘기(Bolbbalgan4)의 '여행(Travel)'을 추천했습니다. 이후 사용자가 자신의 이름을 두 번 물었고, AI는 두 번 모두 '김일남'이라고 정확하게 답변했습니다." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--24faa9ba-0c3d-4785-9cad-17248ce9072a-0' usage_metadata={'input_tokens': 133, 'output_tokens': 827, 'total_tokens': 960, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 746}}


AIMessage(content='볼빨간사춘기는 **여성** 듀오(현재는 안지영 님 솔로 활동)입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--73bb97de-17d0-4a27-8a25-317ce5945c58-0', usage_metadata={'input_tokens': 126, 'output_tokens': 116, 'total_tokens': 242, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 89}})